In [1]:
# import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
mpl.rcParams["figure.dpi"] = 150
import seaborn as sns
import os

# enable copy on write (default in pandas 3.0)
pd.options.mode.copy_on_write = True

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [4]:
merge_meso_2019 = pd.read_csv('../merged/merged_meso_2019.csv', parse_dates=['DATE'])

In [5]:
del merge_meso_2019['TVS_max']

In [6]:
outage = merge_meso_2019[merge_meso_2019['power_outage']==True]
no_outage = merge_meso_2019[merge_meso_2019['power_outage']==False]

In [7]:
merge_meso_2019['y'] = 0

merge_meso_2019.loc[merge_meso_2019.power_outage == True, 'y']=1

In [8]:
merge_meso_2019.sample(5)

,Unnamed: 0,index,DATE,LAT_mean,LON_mean,STR_RANK_max,LL_ROT_VEL_max,LL_DV_max,LL_BASE_max,DEPTH_KFT_max,DPTH_STMRL_max,MAX_RV_KFT_max,MAX_RV_KTS_max,MSI_max,county,state,power_outage,y
143126,143126,113462,2019-03-13,30.491070,-101.930030,5,36,26,11,18,96,11,36,3054,Pecos County,Texas,False,0
882015,882015,830040,2019-12-23,33.665450,-79.670450,5L,42,43,9,10,100,9,42,3508,Williamsburg County,South Carolina,False,0
128974,128974,109924,2019-03-13,31.594642,-99.737812,7,49,72,16,19,100,20,73,5621,Concho County,Texas,False,0
383200,383200,346088,2019-05-22,35.089280,-117.032950,7L,69,87,2,4,100,2,69,5237,San Bernardino County,California,False,0
169292,169292,137519,2019-03-30,38.683280,-90.556850,5L,30,32,1,1,27,2,40,4550,Saint Louis County,Missouri,False,0


In [9]:
merge_meso_2019['DATE'] = pd.to_datetime(merge_meso_2019['DATE'])
merge_meso_2019['Month'] = merge_meso_2019['DATE'].dt.month

In [10]:
all_features =([merge_meso_2019.columns[3], 
                merge_meso_2019.columns[4]] +
                merge_meso_2019.columns[6:14].tolist() +
                [merge_meso_2019.columns[18]])

In [11]:
all_features

['LAT_mean',
 'LON_mean',
 'LL_ROT_VEL_max',
 'LL_DV_max',
 'LL_BASE_max',
 'DEPTH_KFT_max',
 'DPTH_STMRL_max',
 'MAX_RV_KFT_max',
 'MAX_RV_KTS_max',
 'MSI_max',
 'Month']

In [12]:
meso_train, meso_test = train_test_split(merge_meso_2019.copy(),
                                              shuffle=True,
                                              random_state=123,
                                              test_size=.2,
                                              stratify=merge_meso_2019.y.values)

In [13]:
meso_tt, meso_val = train_test_split(meso_train.copy(),
                                              shuffle=True,
                                              random_state=123,
                                              test_size=.2,
                                              stratify=meso_train.y.values)

In [14]:
outage = meso_tt[meso_tt['power_outage']==True]
no_outage = meso_tt[meso_tt['power_outage']==False]
no_outage= no_outage.sample(n=len(outage), random_state=101)
meso_tt_balanced = pd.concat([outage,no_outage],axis=0)

In [17]:
n_splits=5
kfold = StratifiedKFold(n_splits,
                           shuffle=True,
                           random_state=216)

In [18]:
neighbors = range(1, 56)

pca_2_accs = np.zeros((n_splits, len(neighbors)))

pca_2_reccs = np.zeros((n_splits, len(neighbors)))

pca_2_precis = np.zeros((n_splits, len(neighbors)))

# Note:  switching to using "enumerate" from this point in the bootcamp forward.
for i,(train_index, test_index) in enumerate(kfold.split(meso_tt_balanced, meso_tt_balanced.y)):
    print("CV Split", i)
    meso_bal_tt = meso_tt_balanced.iloc[train_index]
    meso_ho = meso_tt_balanced.iloc[test_index]
    
    ## Note, putting the PCA here speeds up the loop
    pca_pipe = Pipeline([('scale', StandardScaler()),
                               ('pca', PCA(2))])
    
    
    pca_tt = pca_pipe.fit_transform(meso_bal_tt[all_features].values)
    pca_ho = pca_pipe.transform(meso_ho[all_features].values)
    
    
    for j, n_neighbors in enumerate(neighbors):
        # No need to scale knn first since PCA is handling that
        knn = KNeighborsClassifier(n_neighbors)
        
        knn.fit(pca_tt, meso_bal_tt.y.values)
        
        pred = knn.predict(pca_ho)
        
        pca_2_accs[i,j] = accuracy_score(meso_ho.y.values, pred)

        pca_2_reccs[i,j] = recall_score(meso_ho.y.values, pred)

        pca_2_precis[i,j] = precision_score(meso_ho.y.values, pred)

CV Split 0
CV Split 1
CV Split 2
CV Split 3
CV Split 4


In [19]:
print(f'The best mean CV accuracy of {np.max(np.mean(pca_2_accs, axis=0)):.3f} was achieved with k = {neighbors[np.argmax(np.mean(pca_2_accs, axis=0))]}' )

The best mean CV accuracy of 0.618 was achieved with k = 55


In [20]:
print(f'The best mean CV recall of {np.max(np.mean(pca_2_reccs, axis=0)):.3f} was achieved with k = {neighbors[np.argmax(np.mean(pca_2_reccs, axis=0))]}' )

The best mean CV recall of 0.717 was achieved with k = 55


In [21]:
print(f'The best mean CV precision of {np.max(np.mean(pca_2_precis, axis=0)):.3f} was achieved with k = {neighbors[np.argmax(np.mean(pca_2_precis, axis=0))]}' )

The best mean CV precision of 0.606 was achieved with k = 4


In [28]:
neighbors = range(1, 56)
comps = range(2,6)

pca_accs = np.zeros((n_splits, len(comps), len(neighbors)))

pca_precis = np.zeros((n_splits, len(comps), len(neighbors)))

pca_reccs = np.zeros((n_splits, len(comps), len(neighbors)))

for i,(train_index, test_index) in enumerate(kfold.split(meso_tt_balanced, meso_tt_balanced.y)):
    print("CV Split", i)
    meso_bal_tt = meso_tt_balanced.iloc[train_index]
    meso_ho = meso_tt_balanced.iloc[test_index]
    
    for j, n_comps in enumerate(comps):
        pca_pipe = Pipeline([('scale', StandardScaler()),
                               ('pca', PCA(n_comps))])
    
    
        pca_tt = pca_pipe.fit_transform(meso_bal_tt[all_features].values)
        pca_ho = pca_pipe.transform(meso_ho[all_features].values)
        
        for k, n_neighbors in enumerate(neighbors):
            knn = KNeighborsClassifier(n_neighbors)
            
            knn.fit(pca_tt,
                    meso_bal_tt.y.values)

            pred = knn.predict(pca_ho)

            pca_accs[i,j,k] = accuracy_score(meso_ho.y.values, pred)

            pca_reccs[i,j,k] = recall_score(meso_ho.y.values, pred)

            pca_precis[i,j,k] = precision_score(meso_ho.y.values, pred)

CV Split 0


CV Split 1
CV Split 2
CV Split 3
CV Split 4


In [23]:
max_index = np.unravel_index(np.argmax(np.mean(pca_accs, axis=0), axis=None), 
                                       np.mean(pca_accs, axis=0).shape)


print(f"The pair with the highest AVG CV Accuracy was k = {neighbors[max_index[1]]} and number of components = {comps[max_index[0]]:.1f}")
print(f"The highest AVG CV Accuracy was {np.max(np.mean(pca_accs, axis=0)):.3f}")

The pair with the highest AVG CV Accuracy was k = 16 and number of components = 4.0
The highest AVG CV Accuracy was 0.766


In [24]:
max_index = np.unravel_index(np.argmax(np.mean(pca_reccs, axis=0), axis=None), 
                                       np.mean(pca_reccs, axis=0).shape)


print(f"The pair with the highest AVG CV recall was k = {neighbors[max_index[1]]} and number of components = {comps[max_index[0]]:.1f}")
print(f"The highest AVG CV recall was {np.max(np.mean(pca_reccs, axis=0)):.3f}")

The pair with the highest AVG CV recall was k = 17 and number of components = 4.0
The highest AVG CV recall was 0.814


In [25]:
max_index = np.unravel_index(np.argmax(np.mean(pca_precis, axis=0), axis=None), 
                                       np.mean(pca_precis, axis=0).shape)


print(f"The pair with the highest AVG CV precision was k = {neighbors[max_index[1]]} and number of components = {comps[max_index[0]]:.1f}")
print(f"The highest AVG CV precision was {np.max(np.mean(pca_precis, axis=0)):.3f}")

The pair with the highest AVG CV precision was k = 2 and number of components = 4.0
The highest AVG CV precision was 0.776


In [26]:
n_splits = 5

## Make the kfold object
kfold = StratifiedKFold(n_splits, 
                        random_state=2013, 
                        shuffle=True)


## the values of C you will try
Cs = [.01, .1, 1, 10, 25, 50, 75, 100, 125, 150]

## this will hold the CV accuracies
C_accs1 = np.zeros((n_splits, len(Cs)))

C_reccs1 = np.zeros((n_splits, len(Cs)))

C_precis1 = np.zeros((n_splits, len(Cs)))


## the cross-validation
for i,(train_index, test_index) in enumerate(kfold.split(meso_tt_balanced, meso_tt_balanced.y)):
    print("CV Split", i)
    meso_bal_tt = meso_tt_balanced.iloc[train_index]
    meso_ho = meso_tt_balanced.iloc[test_index]
    
    for j,C in enumerate(Cs):
        pipe = Pipeline([('scale', StandardScaler()),
                            ('svm', SVC(C=C))])
    
        pipe.fit(meso_bal_tt[all_features],
                    meso_bal_tt.y)
    
        pred = pipe.predict(meso_ho[all_features])

        C_accs1[i, j] = accuracy_score(meso_ho.y, pred)
        C_reccs1[i, j] = recall_score(meso_ho.y, pred)
        C_precis1[i, j] = precision_score(meso_ho.y, pred)

CV Split 0


CV Split 1
CV Split 2
CV Split 3
CV Split 4


In [29]:
mean_cv_accuracy = np.mean(C_accs1, axis=0)
optimal_index = np.argmax(mean_cv_accuracy)
optimal_C = Cs[optimal_index]
optimal_accuracy = mean_cv_accuracy[optimal_index]

print(f"The optimal C was {optimal_C} which gave a mean CV accuracy of {optimal_accuracy:.3f}")

The optimal C was 150 which gave a mean CV accuracy of 0.842


In [30]:
mean_cv_recall = np.mean(C_reccs1, axis=0)
optimal_recall_index = np.argmax(mean_cv_recall)
optimal_recall_C = Cs[optimal_recall_index]
optimal_recall = mean_cv_recall[optimal_recall_index]

print(f"The optimal C was {optimal_recall_C} which gave a mean CV recall of {optimal_recall:.3f}")

The optimal C was 150 which gave a mean CV recall of 0.886


In [31]:
mean_cv_precision = np.mean(C_precis1, axis=0)
optimal_precision_index = np.argmax(mean_cv_precision)
optimal_precision_C = Cs[optimal_precision_index]
optimal_precision = mean_cv_precision[optimal_precision_index]

print(f"The optimal C was {optimal_precision_C} which gave a mean CV precision of {optimal_precision:.3f}")

The optimal C was 150 which gave a mean CV precision of 0.815
